***
# Car prices
***

1. Répertorier l'ensemble des marques de véhicules
2. Répertorier l'ensemble des modèles de véhicules
3. Répertorier l'ensemble des prix des véhicules
***

In [199]:
# Module that sets paths
import path

path_data =     path.path_data
path_df =       path.path_df
path_output =   path.path_output

***

In [200]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import warnings
from datetime import date
import re

requests.packages.urllib3.disable_warnings()

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_colwidth', 1000) 

In [201]:
# ====== Connect to url ====== #
link_original        = 'https://www.guideautoweb.com'
link_constructeur   = 'https://www.guideautoweb.com/constructeurs/'
response_url = requests.get(link_constructeur, verify=False)

if response_url.status_code == 200:
    print('Connection réussi')
else :
    print('Erreur de connexion')

Connection réussi


In [213]:
# ====== 1ere page : Constructeur ====== #
# Extraire toutes les marques et tous leurs liens 

# Get the data from url
html_constructeur = response_url.text
soup_constructeur = BeautifulSoup(html_constructeur, 'html.parser')

# Extraire les marques de la page principale
ul_element = soup_constructeur.find('ul', id='brands-index-list')

data_marque = {
    'marque' : [],
    'link_model' : []
}

# Extract information from <a> elements within the <ul> element
for a_element in ul_element.find_all('a'):
    a_text = a_element.get_text().strip()
    a_href = a_element.get('href')
    link_model = link_original + a_href

    data_marque['marque'].append(a_text)
    data_marque['link_model'].append(link_model)

df = pd.DataFrame(data_marque)
df.head(3)

,marque,link_model
0,Acura,https://www.guideautoweb.com/constructeurs/acura/
1,Alfa Romeo,https://www.guideautoweb.com/constructeurs/alfa-romeo/
2,Allard,https://www.guideautoweb.com/constructeurs/allard/


In [229]:
# ====== 2e page : Modèle====== #
# Extraire tous les modèles et leurs liens

# Loop pour Extraire modèle de la seconde page
count = 0
nombre_iteration = 1

data_model = {
    'model' : [],
    'link_model' : [],
    'production' : []
    }

for link in df['link_model']:
    # if count >= nombre_iteration:
    #     break 

    model_url = requests.get(link, verify=False)
    count += 1
    html_model = model_url.text
    soup_model = BeautifulSoup(html_model, 'html.parser')



    # Extract model en production
    section_EnProduction = soup_model.find_all('div', class_='s')

    extracted_text_list = []
    extracted_ref_list = []

    for section in section_EnProduction:
        try:                                                        # Some model have no element, that cause an error
            a_elements = section.find_all('a', class_='e-a e-t')

            for a_element in a_elements:
                extracted_text_list.append(a_element.get_text())
                extracted_ref_list.append(a_element.get('href'))

        except AttributeError:
            continue

    for text, ref in zip(extracted_text_list, extracted_ref_list):
        data_model['model'].append(text)
        data_model['link_model'].append(link_original + ref)
        data_model['production'].append(True)




    # Extracting model non en production
    section_production_autre = soup_model.find('ul', class_='eg eg-t1 eg-sz-s')
    if section_production_autre:
        for a_element in section_production_autre.find_all('a', class_='txt'):
            try:
                text = a_element.get_text()
                data_model['model'].append(text)
                data_model['link_model'].append(link_original + a_element.get('href'))
                data_model['production'].append(False)
                
            except AttributeError:
                continue

    df_model = pd.DataFrame(data_model)

# df_model.to_excel(path_output + 'df_model.xlsx')

df_model.head(10)     

,model,link_model,production
0,Acura Integra,https://www.guideautoweb.com/constructeurs/acura/integra/2023/,True
1,Acura MDX,https://www.guideautoweb.com/constructeurs/acura/mdx/2023/,True
2,Acura RDX,https://www.guideautoweb.com/constructeurs/acura/rdx/2023/,True
3,Acura TLX,https://www.guideautoweb.com/constructeurs/acura/tlx/2023/,True
4,Acura ZDX,https://www.guideautoweb.com/constructeurs/acura/zdx/2013/,True
5,Acura CL,https://www.guideautoweb.com/constructeurs/acura/cl/,False
6,Acura Concept,https://www.guideautoweb.com/constructeurs/acura/concept/,False
7,Acura CSX,https://www.guideautoweb.com/constructeurs/acura/csx/,False
8,Acura EL,https://www.guideautoweb.com/constructeurs/acura/el/,False
9,Acura ILX,https://www.guideautoweb.com/constructeurs/acura/ilx/,False


In [230]:
# ====== Extraire le liens vers années et marque et modèle ====== #
df_ModelAndYear = df_model

data_df_ModelAndYear = {
    'link_model_an' : []
}

for index, row in df_ModelAndYear.iterrows():
    AnMod_url = requests.get(row['link_model'], verify=False)
    soup_AnMod = BeautifulSoup(AnMod_url.text, 'html.parser')

    annee_list = []

    h1_section = soup_AnMod.find('h1', class_='st st-s3')
    
    for option in h1_section.find_all('option'):
        data_df_ModelAndYear['link_model_an'].append(link_original + option.get('value'))     

df_model2 = pd.DataFrame(data_df_ModelAndYear)

df_model2
# 11m

,link_model_an
0,https://www.guideautoweb.com/constructeurs/acura/integra/2023/
1,https://www.guideautoweb.com/constructeurs/acura/mdx/2023/
2,https://www.guideautoweb.com/constructeurs/acura/mdx/2022/
3,https://www.guideautoweb.com/constructeurs/acura/mdx/2020/
4,https://www.guideautoweb.com/constructeurs/acura/mdx/2019/
...,...
85,https://www.guideautoweb.com/constructeurs/acura/tsx/2012/
86,https://www.guideautoweb.com/constructeurs/acura/tsx/2011/
87,https://www.guideautoweb.com/constructeurs/acura/tsx/2010/
88,https://www.guideautoweb.com/constructeurs/acura/tsx/2009/


In [231]:
# ====== lien vers la table specification ====== #

# spec_tab = df_model2

spec_tab_dic = {
    'spec_tab' : []
}

for index, row in df_model2.iterrows():
    # print(row['link_model_an'])
    tab_url = requests.get(row['link_model_an'], verify=False)
    soup_tab = BeautifulSoup(tab_url.text, 'html.parser')

    tab = soup_tab.find_all('a', class_='tab')
    tab_link = tab[2].get('href').split('/')      # toujours le 3e élément car il y a 4 onglets
    url_total = row['link_model_an'] + tab_link[5] + '/' + tab_link[6] + '/' 
    spec_tab_dic['spec_tab'].append(url_total)
    
df_tab = pd.DataFrame(spec_tab_dic)
df_tab
# 32m   

,spec_tab
0,https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/base/
1,https://www.guideautoweb.com/constructeurs/acura/mdx/2023/specifications/base/
2,https://www.guideautoweb.com/constructeurs/acura/mdx/2022/specifications/base/
3,https://www.guideautoweb.com/constructeurs/acura/mdx/2020/specifications/base/
4,https://www.guideautoweb.com/constructeurs/acura/mdx/2019/specifications/base/
...,...
85,https://www.guideautoweb.com/constructeurs/acura/tsx/2012/specifications/man/
86,https://www.guideautoweb.com/constructeurs/acura/tsx/2011/specifications/man/
87,https://www.guideautoweb.com/constructeurs/acura/tsx/2010/specifications/man/
88,https://www.guideautoweb.com/constructeurs/acura/tsx/2009/specifications/man/


In [232]:
# ====== lien vers les specifications (Trim) ====== #

spec_link_dict = {
    'spec_link' : []
}

for spec_link in df_tab['spec_tab']:
    spec_url = requests.get(spec_link, verify=False)
    soup_spec = BeautifulSoup(spec_url.text, 'html.parser')

    select_section = soup_spec.find('select', {'name': 'trim'})

    for option in select_section.find_all('option'):
        spec_link_dict['spec_link'].append(link_original + option.get('value'))

spec_link_df = pd.DataFrame(spec_link_dict)

spec_link_df
# 29

,spec_link
0,https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/base/
1,https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/a-spec/
2,https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/a-spec-man/
3,https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/elite-a-spec-auto/
4,https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/elite-a-spec-man/
...,...
349,https://www.guideautoweb.com/constructeurs/acura/tsx/2009/specifications/technologie-auto/
350,https://www.guideautoweb.com/constructeurs/acura/tsx/2008/specifications/man/
351,https://www.guideautoweb.com/constructeurs/acura/tsx/2008/specifications/auto/
352,https://www.guideautoweb.com/constructeurs/acura/tsx/2008/specifications/navigation-man/


In [245]:
# ====== Prendre toute l'info pertinante ====== #
import numpy as np
# fonction pour extraire l'info
def extraire_info(string_input):
    try:
        row = info_table.find('th', string=string_input).find_parent('tr')
        value = row.find('td').get_text(strip=True).replace('\xa0', '')
        # print(f'{string_input} : ', value)
        return value
    except AttributeError:
        return np.nan


# Créer un dictionnaire pour storer l'info extraite
list_col_name = ['pdsf', 'moteur', 'type_carbu', 'type_vehicule', 'classement', 'eff_ville', 'eff_autoroute' , 'eff_combi', 'eff_autonomie', 'eff_ges', 'masse']
list_value_extract = ['PDSF', 'Moteur', 'Type de carburant', 'Type de véhicule', 'Classement', 'Ville', 'Autoroute' , 'Combinée', 'Autonomie', 'Émissions CO₂', 'Poids']

dict_info = {}
for element in list_col_name:
    dict_info[element] = []
dict_info['lien']= []

info_link = spec_link_df.copy()

for link in info_link['spec_link']:
    info_url = requests.get(link, verify=False)
    soup_info = BeautifulSoup(info_url.text, 'html.parser')

    # Table qui contient toute l'info
    info_table = soup_info.find('table', {'class': 'dt dt-s1'})

    for key,  element in zip(list_col_name, list_value_extract):
        dict_info[key].append(extraire_info(element))

    dict_info['lien'] = link


df_info = pd.DataFrame(dict_info)
df_info.head()

,pdsf,moteur,type_carbu,type_vehicule,classement,eff_ville,eff_autoroute,eff_combi,eff_autonomie,eff_ges,masse,lien
0,34350$,4L 1.5l,Super,Hatchback,Sous-compacte de luxe,"7,9L/100km","6,3L/100km","7,2L/100km",638km,167g/km,1402kg (3091lb),https://www.guideautoweb.com/constructeurs/acura/tsx/2008/specifications/navigation-auto/
1,37050$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,1L/100km","6,5L/100km","7,4L/100km",621km,172g/km,1415kg (3120lb),https://www.guideautoweb.com/constructeurs/acura/tsx/2008/specifications/navigation-auto/
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.guideautoweb.com/constructeurs/acura/tsx/2008/specifications/navigation-auto/
3,42550$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,1L/100km","6,5L/100km","7,4L/100km",621km,172g/km,1434kg (3161lb),https://www.guideautoweb.com/constructeurs/acura/tsx/2008/specifications/navigation-auto/
4,42550$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,9L/100km","6,5L/100km","7,8L/100km",589km,181g/km,1399kg (3084lb),https://www.guideautoweb.com/constructeurs/acura/tsx/2008/specifications/navigation-auto/


In [ ]:
# df_info.to_hdf(path_df + 'df_info.h5', key='s')

In [211]:
# ====== Nettoyage ====== #
col_dict = {
    'marque' : 4,
    'modele' : 5,
    'annee' : 6,
    'spec' : 8
} 

df_info['split'] = [x.split('/') for x in df_info['lien']]

for key, value in col_dict.items():
    df_info[key] = [x[value] for x in df_info['split']]


order_columns = ['marque', 'modele', 'annee', 'spec', 'pdsf', 'moteur', 'type_carbu', 'type_vehicule', 'classement', 'eff_ville', 'eff_autoroute', 'eff_combi', 'eff_autonomie', 'eff_ges', 'masse', 'lien']
df_info = df_info[order_columns]
df_info

,marque,modele,annee,spec,pdsf,moteur,type_carbu,type_vehicule,classement,eff_ville,eff_autoroute,eff_combi,eff_autonomie,eff_ges,masse,lien
0,acura,integra,2023,base,34350$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"7,9L/100km","6,3L/100km","7,2L/100km",638km,167g/km,1402kg (3091lb),https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/base/
1,acura,integra,2023,a-spec,37050$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,1L/100km","6,5L/100km","7,4L/100km",621km,172g/km,1415kg (3120lb),https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/a-spec/
2,acura,integra,2023,a-spec-man,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/a-spec-man/
3,acura,integra,2023,elite-a-spec-auto,42550$,4L 1.5l,Super,Hatchback,Sous-compacte de luxe,"8,1L/100km","6,5L/100km","7,4L/100km",621km,172g/km,1434kg (3161lb),https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/elite-a-spec-auto/
4,acura,integra,2023,elite-a-spec-man,42550$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,9L/100km","6,5L/100km","7,8L/100km",589km,181g/km,1399kg (3084lb),https://www.guideautoweb.com/constructeurs/acura/integra/2023/specifications/elite-a-spec-man/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28307,volvo,xc70,2010,familiale-5-portes-3-2l-premium,49995$,NaN,NaN,Familiale,n.d.,"13,8L/100km","8,9L/100km","11,6L/100km",NaN,267g/km,1892kg (4171lb),https://www.guideautoweb.com/constructeurs/volvo/xc70/2010/specifications/familiale-5-portes-3-2l-premium/
28308,volvo,xc70,2010,familiale-5-portes-3-0-l-t6,55995$,"6L 3,0l","Ordinaire, Super",Familiale,n.d.,"13,0L/100km","9,0L/100km","11,2L/100km",625km,258g/km,1892kg (4171lb),https://www.guideautoweb.com/constructeurs/volvo/xc70/2010/specifications/familiale-5-portes-3-0-l-t6/
28309,volvo,xc70,2009,familiale-5-portes-moteur-3-2-l,44095$,NaN,NaN,Familiale,n.d.,"13,7L/100km","8,8L/100km","11,5L/100km",NaN,n.d.,1892kg (4171lb),https://www.guideautoweb.com/constructeurs/volvo/xc70/2009/specifications/familiale-5-portes-moteur-3-2-l/
28310,volvo,xc70,2009,familiale-5-portes-3.2l-avec-toit-ouvrant,45595$,NaN,NaN,Familiale,n.d.,"13,7L/100km","8,8L/100km","11,5L/100km",NaN,n.d.,1892kg (4171lb),https://www.guideautoweb.com/constructeurs/volvo/xc70/2009/specifications/familiale-5-portes-3.2l-avec-toit-ouvrant/


In [212]:
# ====== Sauveguarde ====== #
# df_info.to_hdf(path_df + 'scrap_data.h5', key='s')
# df_info.to_excel(path_output + 'scrap_data.xlsx')